In [ ]:
# Matriz: https://sparse.tamu.edu/Janna/Hook_1498

In [27]:
# Codigo basado en el algoritmo propuesto en 
# Shewchuk, J. R. (1994). An Introduction to the Conjugate Gradient Method Without the Agonizing Pain [Technical Report]. Carnegie Mellon University. 

import numpy as np
from scipy.io import mmread
import time

def gradiente_conjugado(A, b, x0=None, imax=300000, eps=1e-8):
    """
    Método del gradiente conjugado para resolver A x = b

    Parámetros:
        A : matriz simétrica definida positiva
        b : vector del lado derecho
        x0 : vector inicial
        imax : número máximo de iteraciones
        eps : tolerancia relativa para el criterio de parada

    Retorna:
        x : solución aproximada
        i : número de iteraciones realizadas
    """
    n = len(b)
    x = np.zeros(n) if x0 is None else x0.copy()

    r = b - A @ x
    d = r.copy()
    delta_new = r @ r
    delta_0 = delta_new
    i = 0

    while i < imax and delta_new > (eps**2) * delta_0:
        q = A @ d
        alpha = delta_new / (d @ q)
        x = x + alpha * d

        if i % 50 == 0:
            r = b - A @ x
        else:
            r = r - alpha * q

        delta_old = delta_new
        delta_new = r @ r
        beta = delta_new / delta_old
        d = r + beta * d
        i += 1

    if delta_new <= (eps**2) * delta_0:
        print(f"Convergió en {i} iteraciones (criterio alcanzado).")
    else:
        print(f"No convergió tras {i} iteraciones (residuo final = {delta_new:.2e}).")

    return x, i


In [28]:
A = mmread("Hook_1498.mtx").tocsr()

n = A.shape[0]

x_true = np.random.rand(n)

b = A @ x_true

x0 = np.zeros_like(b).ravel()
start_time = time.time()
x, iters = gradiente_conjugado(A, b, x0)
print(f"--- {time.time() - start_time:.2f} seconds ---")

error_norm = np.linalg.norm(x - x_true)
print(f"Solution error norm: {error_norm:.2e}")

Convergió en 7319 iteraciones (criterio alcanzado).
--- 397.58 seconds ---
Solution error norm: 2.81e-03


In [29]:
print(x)

[0.32251031 0.24929954 0.69402864 ... 0.68587618 0.10977845 0.82241116]


In [30]:
print(x_true)

[0.32251069 0.24929983 0.69402947 ... 0.6858859  0.10977877 0.82242293]


In [34]:
from scipy.sparse.linalg import cg


def iteration_callback(xk):
    iteration_callback.counter += 1

iteration_callback.counter = 0

n = A.shape[0]
x_true = np.random.rand(n)
b = A @ x_true

start_time = time.time()
x, info = cg(A, b, rtol=1e-8, callback=iteration_callback)
print(f"--- {time.time() - start_time:.2f} seconds ---")

if info == 0:
    print(f"Convergió en {iteration_callback.counter} iteraciones.")
else:
    print(f"No convergió tras {info} iteraciones.")


error_norm = np.linalg.norm(x - x_true)
print(f"Solution error norm: {error_norm:.2e}")


--- 390.39 seconds ---
Convergió en 7303 iteraciones.
Solution error norm: 2.79e-03


In [35]:
print(x)

[0.07947457 0.44071625 0.28981075 ... 0.28275897 0.52333854 0.668301  ]


In [36]:
print(x_true)

[0.07947467 0.44071677 0.2898111  ... 0.28276857 0.52333897 0.66831264]


In [39]:
# Source code
# https://neurontist.hashnode.dev/solving-systems-with-python-discovering-gaussian-elimination-in-machine-learning

import numpy as np

def augmented_matrix(A, B):
    # Horizontally stack A and B to form the augmented matrix
    M = np.hstack((A, B))
    return M

    
def swap(M, index_1, index_2):
    # Swap the rows at index_1 and index_2
    M[[index_1, index_2]] = M[[index_2, index_1]]
    return M


def get_non_zero_element_below_zero_from_column(M, starting_row, column):
    M = M.copy()
    column_array = M[starting_row:, column]
    for i, val in enumerate(column_array):
    # To check for non-zero values, you must always use np.isclose instead of doing "val == 0".
        if not np.isclose(val, 0, atol=1e-5):
            index = i + starting_row
            return index
    return -1


def row_echelon_form(A, B):
    A = A.astype('float64')
    B = B.astype('float64')
    M = augmented_matrix(A, B)
    num_rows = len(A)

    for row in range(num_rows):
        pivot = M[row, row]
        if np.isclose(pivot, 0, atol=1e-5):
            # Swap with a non-zero row
            swap_row = get_non_zero_element_below_zero_from_column(M, row, row)
            if swap_row == -1:
                return 'Singular Matrix'
            M = swap(M, row, swap_row)
            pivot = M[row, row]

        M[row] = M[row] / pivot  # Normalize pivot row
        for j in range(row + 1, num_rows):
            factor = M[j, row]
            M[j] -= factor * M[row]

    return M



def back_substitution(M):
    num_rows = M.shape[0]
    solution = np.zeros(num_rows)

    for row in reversed(range(num_rows)):
        solution[row] = M[row, -1] - np.sum(M[row, :-1] * solution)

    return solution


def gaussian_elimination(A, B):
    M = row_echelon_form(A, B)
    if isinstance(M, str):  # Check if the matrix is singular
        return M
    solution = back_substitution(M)
    return solution

A = A.toarray()
solution = gaussian_elimination(A, b)


MemoryError: Unable to allocate 16.3 TiB for an array with shape (1498023, 1498023) and data type float64